# Communities and Access Restrictions

This notebook is designed to guide you through the process of working with communities and managing access restrictions in Zenodo.

1. **Communities in Zenodo**
   - Understanding the concept of communities
   - Creating and managing communities
   - Assigning records to communities using the Zenodo Toolbox

2. **Access Restrictions**
   - Open Access: Making your research freely available
   - Restricted Access: Controlling who can view your data
   - Embargoed Access: Setting time-based restrictions on your research

3. **Sharing and Accessing Data**
   - Methods for accessing and sharing data, even when it's not public
   - Understanding the difference between private records and private files

In [ ]:
import json
import os
from pathlib import Path
import requests

os.chdir(Path().absolute().parent) if Path().absolute().name == "Tutorials" else None
from db_tools import get_row, initialize_db, print_table
from main_functions import publish_record, retrieve_by_concept_recid, update_metadata
from utilities import load_config, load_json, printJSON

# Initial Upload Configuration
USE_SANDBOX = True
ZENODO_BASE_URL = "https://sandbox.zenodo.org" if USE_SANDBOX else "https://zenodo.org"
USE_ENV_API_KEY = True

if USE_ENV_API_KEY:
    ZENODO_API_KEY = os.environ.get("ZENODO_SANDBOX_API_KEY") if USE_SANDBOX else os.environ.get("ZENODO_API_KEY")
else:
    ZENODO_API_KEY = "your_sandbox_api_key_here" if USE_SANDBOX else "your_production_api_key_here"

HEADERS = {"Content-Type": "application/json"}
PARAMS = {"access_token": ZENODO_API_KEY}


# Initialize Database Connection to track and update Operations
db_config = load_config("Configs/db_config.yaml")
db_path = "Tutorials/sandbox.db"
db_config["local_db_path"] = db_path
db_connection = initialize_db(db_config)

if db_connection:
    print(f"Database connection initialized successfully at {db_path}.")
else:
    print("Failed to initialize database.")
    

# Load record response from database
data = load_json("Tutorials/Output/sandbox_published.json")[-1]
if data:
    print(f"Successfully loaded Zenodo response data with ConceptRecordID {data['conceptrecid']} and RecordID {data['id']} (Version = {data['metadata']['version']})")

**Important:** You will notice that the loaded Response Data has the **Version 0.0.1**. If you have just completed the previous Notebook #04, your latest version should be higher.
<br>To retrieve the response data of the latest version, use `get_row()` for the table `responses`:

In [ ]:
# Retrieve Response Data from Local Database
row_data = get_row(db_connection, "responses", "concept_recid", data["conceptrecid"], print_result=False)
# Parse JSON String to Data
data = json.loads(row_data["data"])
# Print JSON Data of latest Zenodo Response for given ConceptRecordID
printJSON(data)

Alternatively, or if you have created a completely new Record, you can retrieve the latest available record directly from Zenodo:

In [ ]:
latest_msg, latest_data = retrieve_by_concept_recid(data["conceptrecid"], all_versions=False)
data = latest_data[0]
printJSON(data)

## Create and Manage Community

Creating and managing a community on Zenodo is an excellent way to group related research outputs and increase their visibility. Let's explore how to create a community and what settings are available.

### Creating a Community

1. Log in to your Zenodo account at [https://sandbox.zenodo.org/](https://sandbox.zenodo.org/)
2. Navigate to [https://sandbox.zenodo.org/communities/](https://sandbox.zenodo.org/communities/)
3. Click on the "New community" button

### Community Settings

When creating a community, you'll need to provide the following information:

- **Identifier**: A unique identifier for your community (e.g., "my-research-group")
- **Title**: The name of your community
- **Description**: A brief description of the community's purpose and scope
- **Curation policy**: Guidelines for what content is acceptable in the community
- **Page**: An optional longer description or homepage for the community
- **Logo**: An image to represent your community (recommended size: 90x90 pixels)

### Additional Settings

- **Restricted**: Choose whether the community is public or restricted
- **Visibility**: Set the community to be listed publicly or hidden
- **Email notifications**: Opt to receive emails about new uploads to the community

### Managing Community Members

As a community creator, you become the owner. You can add other members with different roles:

- **Curator**: Can accept/reject submissions and edit community details
- **Reader**: Can view restricted communities but cannot curate

To add members:
1. Go to your community page
2. Click on "Edit"
3. Navigate to the "Members" tab
4. Use the "Add member" function to invite users by their email address

### Curating Content

Community curators can:
- Review and accept/reject submissions
- Remove records from the community
- Feature specific records on the community page

For more detailed information, visit Zenodo's [Communities guide](https://help.zenodo.org/communities/).

Using the Zenodo Toolbox, we'll explore programmatic ways to assign communities in the following sections.


After setting up your test community and creating an identifier, assign it in the metadata:

In [4]:
community_identifier = "zenodo-toolbox-tutorial"
data["metadata"]["communities"] = [{'identifier': community_identifier}]

### Push Metadata to Public Record
In the next step, we have to push the new metadata in order to assign it to the desired community.
<br>To modify an existing, already published Record, we need to set it into 'Edit Mode' first, and then push the Metadata.
<br>In order to publish it correctly after all changes are made, we additionally need to remove the DOI from the metadata that is being pushed to the existing record.

<small>

Note: The manual removal of the DOI from the metadata seems to be a current workaround and might change in future. If this is not done, you will receive a 400 Validation Error when trying to publish the changed record.

</small>

In [ ]:
# enable 'Edit Mode' for the public record
r_edit = requests.post(data["links"]["edit"], params=PARAMS)
if not r_edit.status_code == 201:
    print(f"Something went wrong, check Error messages: {r_edit.json()}")

# remove DOI from metadata that shall be pushed
del data["metadata"]["doi"]

# push metadata to record
metadata_msg, metadata_data = update_metadata(data, {"metadata": data["metadata"]}, db_connection)
if metadata_msg["success"]:
    print(f"New Metadata pushed to published record with RecordID {metadata_data['id']}.")
    printJSON(metadata_data)
    print_table(db_connection, "operations", metadata_data["conceptrecid"])
else:
    print(f"Something went wrong, check Error messages: {metadata_msg}")

### Publish Record with updated Metadata

Now we can finally publish the updated changes, and the record should be available as a part of your Zenodo community, if it was approved manually or automatically.

In [ ]:
# Publish Record
publish_msg, publish_data = publish_record(metadata_data, db_connection)

if publish_msg["success"]:
    print("Record successfully published!")
    print(f"DOI: {publish_data['doi']}")
    print(f"Record URL: {publish_data['links']['record_html']}")
    print_table(db_connection, "operations", metadata_data["conceptrecid"])
else:
    print("Failed to publish record. Error message:")
    print(publish_msg["text"])

## Access Restrictions and Accessing Restricted Data

Zenodo offers different levels of access restrictions for uploaded datasets and files. Understanding these restrictions is crucial for both data providers and users. The three main types of access restrictions are:

### 1. Open Access

- **Definition**: Files are freely available for anyone to download and use.
- **Visibility**: Metadata and files are publicly visible and downloadable.
- **Use Case**: Ideal for open science initiatives and maximizing research impact.

### 2. Embargoed Access

- **Definition**: Files are temporarily restricted but will become openly accessible after a specified date.
- **Visibility**: Metadata is visible, but files are not downloadable until the embargo lifts.
- **Use Case**: Useful for datasets associated with upcoming publications or patent applications.

### 3. Restricted Access

- **Definition**: Files are only accessible to specific users or groups.
- **Visibility**: Metadata may be visible, but file access is controlled.
- **Use Case**: Suitable for sensitive data, proprietary information, or datasets with ethical considerations.

### Working with Restricted Data

In the following notebook sections, you'll learn two key aspects of handling restricted data on Zenodo:

1. **Making Data Restricted**: You'll discover how to upload files and set appropriate access restrictions using the Zenodo API.

2. **Accessing Restricted Data**: We'll explore methods to programmatically request and retrieve restricted data through the API, with proper authentication.

These practical exercises will equip you with the skills to manage sensitive data effectively on the Zenodo platform, both as a data provider and a data user.
Remember, always respect the access restrictions and usage terms set by the data providers when working with Zenodo datasets.


Let's start by modifying and pushing the Metadata:
1) Set the access right
2) Remove the license from metadata
3) Define access conditions
4) Enter Edit Mode, Remove DOI & Push modified Metadata to existing Record

In [ ]:
data = publish_data # assign latest response data
data["metadata"]["access_right"] = "restricted" # set access_right here
if "license" in list(data["metadata"].keys()): # delete license from metadata after access restriction
    del data["metadata"]["license"]
data["metadata"]["access_conditions"] = "Contact the owner to request an access token for this restricted data." # short explanation on how to acquire access

# Enable Edit Mode
r_edit = requests.post(data["links"]["edit"], params=PARAMS)
if not r_edit.status_code == 201:
    print(f"Something went wrong, check Error messages: {r_edit.json()}")
    
# Remove DOI and push Metadata
del data["metadata"]["doi"]
metadata_msg, metadata_data = update_metadata(data, {"metadata": data["metadata"]}, db_connection)
if metadata_msg["success"]:
    print(f"New Metadata pushed to published record with RecordID {metadata_data['id']}.")
    printJSON(metadata_data)
    print_table(db_connection, "operations", metadata_data["conceptrecid"])
else:
    print(f"Something went wrong, check Error messages: {metadata_msg}")

### Publish Record and Check Access

Let's publish the changes:

In [ ]:
# Publish Record
publish_msg, publish_data = publish_record(metadata_data, db_connection)

if publish_msg["success"]:
    print("Record successfully published!")
    print(f"DOI: {publish_data['doi']}")
    print(f"Record URL: {publish_data['links']['record_html']}")
    print_table(db_connection, "operations", metadata_data["conceptrecid"])
else:
    print("Failed to publish record. Error message:")
    print(publish_msg["text"])

### Check the Record and Access Data

Click on the Record URL printed above, and you should see a red 'Restricted' tag on the upper right of the record page.
<br>Let's try to access the data without an authorized access token, and afterwards with the one we have used all the time.
<br>You already know how to create an access token, so if you want to provide any user with a **read-only** token to protect your records, it should be inserted in the `params`.

<small>

**Important**: As usual, the download links provided in the response still contain the `/draft` breadcrump, which must be removed, else it will return a `404`.

</small>

In [ ]:
data = publish_data
files_data = data["files"]

# Wrong Token Access
access_established = False
test_download_link = files_data[0]["links"]["download"].replace("/draft", "")
r_download = requests.get(test_download_link, params={"access_token": "123456789"})
print(f"Response with wrong Token: {json.dumps(r_download.json(), indent=2)}")

# Authorized Token Access
r_download = requests.get(test_download_link, params={"access_token": ZENODO_API_KEY}) # set custom API key here, e.g. with read-only rights.
if r_download.status_code == 200:
    access_established = True
    print(f"Response with authorized Token: {r_download.status_code}")
else:
    print("Something went wrong. Did you use the correct access token in order to access the restricted data?")

# Download Restricted Files
if access_established:
    savepath = f'Tutorials/Output/{files_data[0]["filename"]}'
    with open(savepath, "wb") as file:
        file.write(r_download.content)
    db_connection.close()
else:
    print("Access to restricted data not established.")

### Embargoed Data

Setting the access_right to `embargoed` basically means that you decide when your data will become openly accessible.
<br>The main differences to the above procedures are that you have to set a `license` and an `embargo_date` in the metadata.

## Conclusion

This notebook has demonstrated key procedures for managing restricted data on Zenodo:

1. Implementing access restrictions during data upload
2. Accessing restricted data via the Zenodo API

These operations are essential for maintaining data confidentiality while facilitating controlled sharing within the scientific community. Users are reminded to adhere to Zenodo's terms of service, data protection regulations, and ethical guidelines when handling restricted datasets.

For further information, please consult the official [Zenodo API documentation](https://developers.zenodo.org/).